In [3]:
import sys
sys.path.append("..")
import numpy as np
import pandas as pd
import quaternion
import re
import datetime
import os
import glob
from MOF_build.functions.rotate import rotate
from MOF_build.functions.read import read
from MOF_build.functions.output import output

In [5]:
'''translate and rotate node to beginning point'''

#node = read.pdb('PCN222_lib_nodeZr.pdb')
node = read.pdb('node_lib.pdb')
#print(node.iloc[30,:],node.iloc[57,:])
array = node.loc[:, ['x','y','z']].to_numpy()
center = 0.5*(array[47]+array[46]) #center point (beginning point)
new_node = array-center

any_tdx, any_tdy, any_tdz = (
np.array([1, 0, 0]),
np.array([0, 1, 0]),
np.array([0, 1, 2]))
dx, dy, dz = np.array([1, 0, 0]), np.array([0, 1, 0]), np.array([0, 0, 1])
carte_basis = np.vstack([dx, dy, dz])
    # for uio66 fcb
tric_basis = rotate.coordinate_transfer(any_tdx, any_tdy, any_tdz, carte_basis)
#v1_file = new_node[9]-new_node[0]
#v2_file = new_node[27]-new_node[18]
v1_file = new_node[47]-new_node[46]
v2_file = new_node[40]-new_node[43]
v1_frame = tric_basis[2]-tric_basis[1]
v2_frame = tric_basis[2]+tric_basis[1]
rotated_node = rotate.rotate_twice_linker(node,center,v1_file,v1_frame,v2_file,v2_frame)


node.loc[:, ['x','y','z']] = rotated_node
newdf = pd.concat([node.iloc[:,0:3],node.iloc[:,6],node.iloc[:,3:6]],axis=1, join = 'outer') #move ['NOTE'] to column3
newdf.to_csv('PCNZRRnodeOO.txt',header=None,sep='\t',index=False)
output('PCNZRRnodeOO',True,False,False)

In [12]:
#reset resnumber  
justnode = read.pdb('PCNZRRnodeOO.pdb') #ZRRnode.pdb
justnode.iloc[:,2]=1 #set resnumber as 1
#print(justnode)
df = pd.concat([justnode.iloc[:,0:3],justnode.iloc[:,6],justnode.iloc[:,3:6]],axis=1, join = 'outer')
df.to_csv('lib_ZRnode.txt',header=None,sep='\t',index=False)
output('lib_ZRnode',True,False,False)


In [13]:
'''add dummy atom to Zr node'''
pre_dummy_pdb = read.pdb('PCNZRRnodeOO.pdb')

atom_type = []
for i in pre_dummy_pdb['Atom_label']:
    atom_type.append(re.sub(r"\d", "", i))
pre_dummy_pdb.insert(7, "atom_type", atom_type, True)

df_Zr =pre_dummy_pdb[pre_dummy_pdb['atom_type']=='Zr'].reset_index()
df_O =pre_dummy_pdb[pre_dummy_pdb['atom_type']=='O'].reset_index()
O_positions = df_O.loc[:, ['x','y','z']].to_numpy()
df_dummy = read.pdb("dummy_template.pdb")
for i in range(df_Zr.shape[0]):
    distance_O = []
    distance_Osort = []
    pointZR = df_Zr.loc[i, ['x','y','z']].to_numpy()
    for j in range(len(O_positions)):
        distance_O.append(np.linalg.norm(pointZR-O_positions[j]) )
        distance_Osort.append(np.linalg.norm(pointZR-O_positions[j]) )
    distance_Osort.sort()
    threshhold = distance_Osort[8]
    indices = [index for index, num in enumerate(distance_O) if num < threshhold]
    #print(indices,distance_O)
    dummy_atoms = np.zeros((8,3))
    row =0
    for k in indices:
        newD = pointZR+(O_positions[k]-pointZR)/np.linalg.norm(pointZR-O_positions[i])
        dummy_atoms[row,:]=newD
        row +=1
    print(pointZR,'\n')
    dummy_atoms=np.round(dummy_atoms,3)
    df_dummy.loc[0, ['x','y','z']]=pointZR
    df_dummy.loc[1:, ['x', 'y', 'z']] = dummy_atoms
    if i <1:
        df_add_dummy = pd.concat([read.pdb('PCNZRRnodeOO.pdb'),df_dummy],ignore_index=True)
    else:
        df_add_dummy = pd.concat([df_add_dummy,df_dummy],ignore_index=True)

#output('0adddummy',True,False,False)


[0.265 -1.303 -2.108] 

[2.477 0.142 0.23] 

[-2.477 -0.142 -0.23] 

[-0.265 1.303 2.108] 

[-0.0 2.091 -1.293] 

[-0.0 -2.091 1.293] 



In [14]:

df_add_dummy.insert(3, "note", df_add_dummy['Note'], True)
df_add_dummy.to_csv('0adddummy.txt',header=None,sep='\t',index=False)
#
output('0adddummy',True,False,False)